In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
# import tensorflow_datasets as tfds
from functools import partial

2024-04-30 14:14:03.602574: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 14:14:03.651628: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 14:14:03.651700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 14:14:03.656278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 14:14:03.674039: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 14:14:03.675160: I tensorflow/core/platform/cpu_feature_guard.cc:1

# MODEL 1.0

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()

In [16]:
X_train, y_train = X_train_full[5000:], y_train_full[5000:]
X_val, y_val = X_train_full[:5000], y_train_full[:5000]
X_train = X_train / 255.
X_val = X_val / 255.
X_test = X_test / 255.

X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_train

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32, kernel_size=3, 
                              padding="SAME", input_shape=[28,28,1]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPooling2D(pool_size=2))

model.add(keras.layers.Conv2D(filters=64, kernel_size=3, 
                              padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Conv2D(filters=64, kernel_size=3, 
                              padding="same"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPooling2D(pool_size=2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(10, activation="softmax"))

2024-04-30 14:14:05.362013: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-30 14:14:05.387463: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
1719/1719 [==============================] - 52s 29ms/step - loss: 0.2645 - accuracy: 0.9199 - val_loss: 0.0595 - val_accuracy: 0.9832
Epoch 2/10
1719/1719 [==============================] - 48s 28ms/step - loss: 0.1153 - accuracy: 0.9664 - val_loss: 0.0470 - val_accuracy: 0.9860
Epoch 3/10
1719/1719 [==============================] - 48s 28ms/step - loss: 0.0829 - accuracy: 0.9756 - val_loss: 0.0317 - val_accuracy: 0.9900
Epoch 4/10
1719/1719 [==============================] - 48s 28ms/step - loss: 0.0696 - accuracy: 0.9805 - val_loss: 0.0446 - val_accuracy: 0.9898
Epoch 5/10
1719/1719 [==============================] - 48s 28ms/step - loss: 0.0573 - accuracy: 0.9831 - val_loss: 0.0312 - val_accuracy: 0.9906
Epoch 6/10
1719/1719 [==============================] - 48s 28ms/step - loss: 0.0498 - accuracy: 0.9859 - val_loss: 0.0319 - val_accuracy: 0.9930
Epoch 7/10
1719/1719 [==============================] - 48s 28ms/step - loss: 0.0434 - accuracy: 0.9871 - val_loss: 0.0303 -

In [6]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.0226 - accuracy: 0.9931


[0.022594379261136055, 0.9930999875068665]

# MODEL 2.0

In [7]:
keras.backend.clear_session()

In [8]:
model_2 = keras.models.Sequential([
#     conv layer 1
    keras.layers.Conv2D(filters=64, kernel_size=5, strides=1, padding="same",
                       input_shape=[28,28,1]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    
    keras.layers.MaxPool2D(pool_size=2, padding="same"),
    keras.layers.Dropout(rate=0.3),
#     conv layer 2
    keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    
    keras.layers.MaxPool2D(pool_size=4, padding="same"),
    keras.layers.Dropout(rate=0.3),
    keras.layers.Flatten(),
    
#     hidden layer 3
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(rate=0.4),
    
    keras.layers.Dense(10, activation="softmax")
    
    
])

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
)

# Reduce learning rate when validation loss plateaus
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5
)

In [10]:
# model_2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
# model_2.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

In [11]:
# model_2.evaluate(X_test,y_test)

In [12]:
# y_pred = np.argmax(model.predict(X_test), axis=1)

# MODEL 3.0 WITH AUGMENTED DATA



In [13]:
keras.backend.clear_session()

In [14]:
from scipy import ndimage
import random

def data_augmentation(images, labels):
    aug_images = []
    aug_labels = []
    
    for image, label in zip(images, labels):
        aug_images.append(image)
        aug_labels.append(label)
        
        # Background image for filling empty pixel
        bg_value = np.median(image)
        for _ in range(4):
            # Rotation
            rot_image = ndimage.rotate(image, angle=random.randint(-15, 15), 
                                       reshape=False, cval=bg_value)
            # Shift
            shift_image = ndimage.shift(rot_image, shift=np.random.randint(-2, 2, 2), 
                                        cval=bg_value)
            
            aug_images.append(shift_image)
            aug_labels.append(label)
    aug_images = np.array(aug_images)
    aug_labels = np.array(aug_labels)
    return aug_images, aug_labels

In [15]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, y_train = data_augmentation(X_train, y_train)

# Convert numpy float type and normalize it
X_train = X_train.astype(np.float32) / 255.
X_test = X_test.astype(np.float32) / 255.

# Convert it to 4D array (or we can use np.expand_dims for dimension expansion)
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

# Build dataset pipeline
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=500000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

KeyboardInterrupt: 

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True,
)

# Reduce learning rate when validation loss plateaus
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5
)

In [ ]:
model_2.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model_2.fit(train_ds, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

In [ ]:
model_2.evaluate(test_ds)